In [ ]:
!pip install transformers
!pip install torch
!pip install ekphrasis
!git clone https://github.com/hate-alert/Tutorial-ICWSM-2021.git

In [ ]:
!cd Tutorial-ICWSM-2021

In [ ]:
import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import re
import random
import torch.nn.functional as F
import numpy as np
from Code.utils import *
from Code.model import *
from Code.predictions import *

In [ ]:
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

In [ ]:
model = modelPredRationale(model_path='Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two',device=device)

In [ ]:
import pandas as pd
dataset = pd.read_csv('https://raw.githubusercontent.com/marcoguerini/CONAN/master/Multitarget-CONAN/Multitarget-CONAN.csv')

In [ ]:
dataset.rename(columns = {'HATE_SPEECH':'Sentences'}, inplace = True)

In [ ]:
dataset.rename(columns = {'INDEX':'Index'}, inplace = True)

In [ ]:
## provided function by HateXplain authors in demo, which I have slightly modified

def getDatasetPrediction(dataset,config):
    labels,attention,sents=model.return_rationales(dataset['Sentences'])
    predictions = {}
    for index,row in dataset.iterrows():
        dict1={}
        dict_labels={}
        for ele in config:
            dict_labels[config[ele]]=round(labels[index][ele],3)

        dict1["Tokens"] = sents[index]
        dict1["Rationale"]=attention[index][0:len(dict1["Tokens"])]
        predictions[row['Index']] = dict1
    return predictions

In [ ]:
config=model.config.id2label
predictions2 = getDatasetPrediction(dataset,config)

In [ ]:
import pickle
f = open("file.pkl","wb")

pickle.dump(predictions2,f)
f.close()